### Imports

In [1]:
import os
os.chdir('/..')


In [6]:
print(os.getcwd())

/


In [2]:
import data.data as data
import inout.importexport as imp
import numpy as np
import pandas as pd
import xgboost as xgb
import random
import math
from pandas.api.types import CategoricalDtype
from scipy.sparse import load_npz
import gc

ModuleNotFoundError: No module named 'data'

### Create the complete dataframe

#### Read the recommendations from csv

In [ ]:
raw_recs = imp.importcsv('reranking/train_als_4coll_xgboost25_23-58-10.csv', check_len=-1)

print(raw_recs[0:2])

#### Explode each row into multiple rows (one per interaction)

In [ ]:
recs_tracks = []
for rec in raw_recs:
    playlist_id = rec[0]
    for t in rec[1:]:
        recs_tracks.append([playlist_id, t])
recs_df = pd.DataFrame(recs_tracks, columns=['playlist_id','track_id'])

print(recs_df)

#### Append the 'profile_length' column to the recommendation dataframe

In [ ]:
target_ids = data.get_target_playlists()
targetURM = data.get_urm_train_1()[target_ids]
user_profile_lengths = np.array(targetURM.sum(axis=1)).flatten()
profile_lengths_df = pd.DataFrame({'playlist_id': target_ids, 'profile_length': user_profile_lengths})

print(profile_lengths_df.head(10))

In [ ]:
rec_lengths_df = recs_df.merge(profile_lengths_df, on='playlist_id')
print(rec_lengths_df)

#### Popularity feature

In [ ]:
df = data.get_playlists_df()
popularity = df.groupby(['track_id']).size().reset_index(name='popularity')
print(popularity)

In [ ]:
rec_pop_df = rec_lengths_df.join(popularity.set_index('track_id'), on='track_id')
print(rec_pop_df)

#### Append the 'label' column 

In [ ]:
urm_test = data.get_urm_test_1()
test_labels = []

last_playlist_id = -1
for idx,row in recs_df.iterrows():
    current_playlist_id = row['playlist_id']
    track_id = row['track_id']
    # cache the row of the urm test if same playlist of the previous iteration
    if not current_playlist_id == last_playlist_id:
        # tracks ids in the t row of urm test
        tracks_ids = urm_test.getrow(current_playlist_id).nonzero()[1]
        last_playlist_id = current_playlist_id
    
    test_labels.append(1 if track_id in tracks_ids else 0)

test_labels_df = pd.DataFrame({'label': test_labels})

In [ ]:
rec_label_df = pd.concat([rec_pop_df, test_labels_df], axis=1)
print(rec_label_df)

In [ ]:
matrix = [load_npz('raw_data/saved_r_hat_evaluation/4collaborative_l2.npz'), load_npz('raw_data/saved_r_hat_evaluation/als_l2.npz')]
scores = [[], []]
n = len(matrix)
for idx,row in rec_label_df.iterrows():
    for i in range(len(matrix)):
        scores[i].append(matrix[i][row['playlist_id'], row['track_id']])

dfn1 = pd.DataFrame({'score_4_coll': scores[0]/max(scores[0])})
dfn2 = pd.DataFrame({'score_als': scores[1]/max(scores[1])})
rec_scores_df = pd.concat([rec_label_df, dfn1, dfn2], axis=1)
print(rec_scores_df)

In [ ]:
del matrix
gc.collect()

#### Append the tracks features (album, artist, duration)

In [ ]:
tdf = data.get_tracks_df()
rec_feature_track_df = rec_scores_df.join(tdf.set_index('track_id'), on='track_id')
print(rec_feature_track_df)

### I'm happy with the features gathered

In [ ]:
full = rec_feature_track_df.copy()

### Split into train and test dataframes

In [ ]:
def func(x):
    n = x['label'].sum()
    ones = x.loc[x['label'] == 1]
    zeros = x.loc[x['label'] == 0].sample(n)
    return pd.concat([ones,zeros])

In [ ]:
tgt = data.get_target_playlists()
train_tgt = random.sample(tgt, math.floor(len(tgt)*0.8))
test_tgt = list(set(tgt) - set(train_tgt))
train = full.loc[full['playlist_id'].isin(train_tgt)]
test = full.loc[full['playlist_id'].isin(test_tgt)]

In [ ]:
train = train.groupby(['playlist_id'], as_index=False).apply(func)
train = train.reset_index().drop(['level_0', 'level_1'], axis=1)

In [ ]:
test

#### One hot encodings

In [ ]:
to_concat_train = []
to_concat_test = []
to_onehot = []
to_drop = ['album_id', 'artist_id', 'track_id', 'playlist_id']

In [ ]:
def onehotize(full, df, str):
    exp = full[str].unique()
    print(len(exp))
    df.loc[:, (str)] = df[str].astype(CategoricalDtype(categories = exp))
    oh = pd.get_dummies(df[str], prefix=str).to_sparse(fill_value=0)
    return oh

In [ ]:
for name in to_onehot:
    to_concat_train.append(onehotize(full, train, name))
    to_concat_test.append(onehotize(full, test, name))
    train = train.drop(name, axis=1)
    test = test.drop(name, axis=1)
to_concat_train.insert(0, train)
to_concat_test.insert(0, test)

In [ ]:
train = pd.concat(to_concat_train, axis=1)
test = pd.concat(to_concat_test, axis=1)

In [ ]:
traind = train.copy()
testd = test.copy()
for j in to_drop:
    traind = traind.drop(j, axis=1)
    testd = testd.drop(j, axis=1)

In [ ]:
traind

In [ ]:
testd

# Train and save the model

In [ ]:
import xgboost as xgb
label_train = traind.label
trainu = traind.drop(['label'], axis=1)
label_test = testd.label
testu = testd.drop(['label'], axis=1)
dtrain = xgb.DMatrix(trainu, label=label_train)
dtest = xgb.DMatrix(testu, label=label_test)

param = {'max_depth': 16, 'eta': 0.1, 'silent': 1, 'objective': 'binary:logistic',
         'lambda': 2, 'alpha': 2, 'tree_method': 'auto', 'grow_policy':'lossguide', 'subsample': 0.5}
param['nthread'] = 4
param['eval_metric'] = 'auc'

evallist = [(dtest, 'eval'), (dtrain, 'train')]

num_round = 100
model_trained = xgb.train(param, dtrain, num_round, evallist)

In [ ]:
model_trained.save_model('0001.model')

In [ ]:
ypred = model_trained.predict(xgb.DMatrix(testu))

In [ ]:
df_preds = pd.DataFrame({'score': ypred})
df_boosted = pd.concat([test, df_preds], axis=1)
def fu(x):
    x = x.sort_values(by=['score'], ascending=False).head(10)
    return np.concatenate((x.playlist_id.unique(), x.track_id.unique())).astype(np.int32)
df_boosted_sorted = df_boosted.groupby(['playlist_id'], as_index=False).apply(fu)
recs = list(df_boosted_sorted.values)

In [ ]:
def evaluate(recommendations, test_urm, at_k=10, verbose=True):
    aps = 0.0
    for r in recommendations:
        row = test_urm.getrow(r[0]).indices
        m = min(at_k, len(row))

        ap = 0.0
        n_elems_found = 0.0
        for j in range(1, m+1):
            if r[j] in row:
                n_elems_found += 1
                ap = ap + n_elems_found/j
        if m > 0:
            ap = ap/m
            aps += ap

    result = aps/len(recommendations)
    if verbose:
        print('MAP: {}'.format(result))

    return result

In [ ]:
evaluate(recs, data.get_urm_test_1())